In [1]:
import geopandas as gpd
import pandas as pd
import fiona
import os
import matplotlib.pyplot as plt
import folium
from zipfile import ZipFile
from folium.plugins import MarkerCluster, HeatMap, BeautifyIcon
from folium.map import LayerControl, Layer, FeatureGroup
import seaborn as sns
from shapely.geometry import Point, LineString, MultiPoint
import numpy as np
import contextily as ctx
import requests
from io import StringIO, BytesIO
import json
import datetime as dt
from ast import literal_eval
from shapely.wkt import loads
import plotly.express as px
import re

In [2]:
root = 'raw_data/police_complaints/'
files = os.listdir(root)

dfs = {}
for file in files:
    if '.csv' in file:
        df = pd.read_csv(f'{root}{file}')
        dfs[f'{file}{df.shape[1]}'] = df
    else:
        df = pd.read_excel(f'{root}{file}')
        dfs[f'{file}{df.shape[1]}'] = df
        
dfs['2019_2.csv1'] = pd.read_csv(f'{root}2019_2.csv', sep=';') #fuckup

In [3]:
all_complaints = pd.concat(dfs.values())
all_complaints['Data'] = pd.to_datetime(all_complaints['Data'])
all_complaints['Sottocategoria Criminologica'].value_counts()

Altro                         3977
Decoro e degrado urbano       2439
Disturbi da locali             768
Rumori molesti                 639
Comportamenti molesti          377
Disturbi Cani                  275
Veicoli abbandonati             72
Uso improprio parti comuni      45
Aggregazioni giovanili          24
Atti di vandalismo              20
Disturbi altri animali          10
Name: Sottocategoria Criminologica, dtype: int64

In [4]:
all_complaints_noise = all_complaints[(all_complaints['Sottocategoria Criminologica'] == 'Rumori molesti')\
                                     | (all_complaints['Sottocategoria Criminologica'] == 'Aggregazioni giovanili')\
                                     | (all_complaints['Sottocategoria Criminologica'] == 'Disturbi da locali')\
                                     | (all_complaints['Sottocategoria Criminologica'] == 'Local disturbances')]
all_complaints_noise.shape


(1431, 7)

In [23]:
complaints_2018 = all_complaints_noise[(all_complaints_noise['Data'] > '2018-01-01') \
                                       & (all_complaints_noise['Data'] < '2018-12-31')]


complaints_2018.to_csv('raw_data/complaints_2018.csv')


In [5]:
all_complaints_noise.Circoscrizione.describe()

count    1417.000000
mean        4.634439
std         2.768446
min         1.000000
25%         2.000000
50%         5.000000
75%         7.000000
max        10.000000
Name: Circoscrizione, dtype: float64

In [6]:
zones = gpd.read_file('raw_data/zone_statistiche_geo')
zones['ZONASTAT'] = zones['ZONASTAT'].apply(lambda x: re.sub("[^0-9]", "", x)).astype(int)

In [7]:
complaints_date = all_complaints_noise.groupby(['Data', 'Circoscrizione']).count()[['Localita']].reset_index()
complaints_shp = complaints_date.merge(zones, 
                                       left_on='Circoscrizione',
                                       right_on='ZONASTAT')

In [8]:
mapable = gpd.GeoDataFrame(complaints_shp.groupby('ZONASTAT').agg({'Localita':'mean',
                                               'geometry':'first'}))
mapable = mapable.set_crs('EPSG:3003')
mapable = mapable.to_crs('EPSG:3857')

In [9]:
mapable

,Localita,geometry
ZONASTAT,,
1,1.250000,"POLYGON ((854420.153 5632875.411, 854470.197 5..."
2,1.037037,"POLYGON ((855400.704 5632952.711, 855446.466 5..."
3,1.133333,"POLYGON ((855189.904 5632505.860, 855233.672 5..."
4,1.096154,"POLYGON ((854345.767 5631838.946, 854395.435 5..."
5,1.049020,"POLYGON ((853633.229 5633395.257, 853646.291 5..."
6,1.110169,"POLYGON ((856059.578 5631808.449, 856095.141 5..."
7,1.135417,"POLYGON ((855106.531 5631252.773, 855231.830 5..."
8,1.153846,"POLYGON ((853012.386 5632317.806, 853087.993 5..."
9,1.017241,"POLYGON ((854593.661 5630534.173, 854610.146 5..."


In [ ]:
# get sensors
sensors = pd.read_csv('raw_data/noise_sensor_list.csv',
                     sep=';',
                     decimal=',')

sensors['geometry'] = sensors.apply(lambda x: Point(x.Lat, x.Long), axis=1)
sensors_geo = gpd.GeoDataFrame(sensors)
sensors_geo = sensors_geo.set_crs(epsg=3857)
# sensors_geo = sensors_geo.to_crs('EPSG:3003')

In [ ]:
sensors_geo

In [ ]:
# map zones
m = folium.Map([45.072433300397435, 7.668045740145775], zoom_start=13)

folium.Choropleth(mapable,
#                  data=mapable,
                 toolip).add_to(m)

for i, row in sensors.iterrows():
    folium.Marker(location=[row.Lat, row.Long]).add_to(m)

m